In [37]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy

In [38]:
def get_conn(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [39]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [40]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'
query = """with ops_main as (
    select
        awb
    ,   pickuptime
    from public.ops_main
    where 1=1
    and pickuptime is not null
    and shipping_partner = 'Hyperlocal'
    and date_trunc('day', pickuptime) = date_trunc('day', now() + interval'5.5 hours')
)
,
shipment_order_details as (
    select
        awb
    ,   warehouse_id
    from public.shipment_order_details
)
,
base as (
    select
        s.warehouse_id
    ,   o.pickuptime
    ,   o.awb
    from ops_main o
    left join shipment_order_details s on o.awb = s.awb
    order by 1, 2
)

select * from base"""

# Establish a connection
conn = get_conn(SSH_required, key_path)

# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame 'df'
# print(df)

Tunnel Started
Connection Made


In [42]:
df['pickuptime'] = pd.to_datetime(df['pickuptime'])

def round_down_to_nearest_10min(dt):
    return dt - timedelta(minutes=dt.minute % 10, seconds=dt.second, microseconds=dt.microsecond)

df['window'] = df['pickuptime'].apply(round_down_to_nearest_10min)

# Generate all possible 10-minute windows
start_time = df['pickuptime'].min().floor('10T')
end_time = df['pickuptime'].max().ceil('10T')
all_windows = pd.date_range(start=start_time, end=end_time, freq='10T')

# Create a DataFrame for windows and initialize as inactive
window_df = pd.DataFrame({'window': all_windows})
window_df['active'] = False

# Add warehouse_id to windows and initialize as inactive
unique_warehouses = df['warehouse_id'].unique()
window_df = window_df.merge(pd.DataFrame({'warehouse_id': unique_warehouses}), how='cross')
window_df['active'] = False

# Mark active windows based on pickups
for warehouse in unique_warehouses:
    warehouse_windows = df[df['warehouse_id'] == warehouse]['window'].unique()
    window_df.loc[(window_df['warehouse_id'] == warehouse) & (window_df['window'].isin(warehouse_windows)), 'active'] = True

# Identify pickup start and end times
window_df['pickup_start'] = False
window_df['pickup_end'] = False

for warehouse in unique_warehouses:
    warehouse_windows = window_df[window_df['warehouse_id'] == warehouse].reset_index(drop=True)
    for i in range(len(warehouse_windows) - 1):
        current_active = warehouse_windows.iloc[i]['active']
        next_active = warehouse_windows.iloc[i + 1]['active']
        if i == 0 and current_active:
            window_df.loc[(window_df['warehouse_id'] == warehouse) & (window_df['window'] == warehouse_windows.iloc[i]['window']), 'pickup_start'] = True
        if not current_active and next_active:
            window_df.loc[(window_df['warehouse_id'] == warehouse) & (window_df['window'] == warehouse_windows.iloc[i + 1]['window']), 'pickup_start'] = True
        if current_active and not next_active:
            if i > 0:  # Ensure it's not the first window of the day
                window_df.loc[(window_df['warehouse_id'] == warehouse) & (window_df['window'] == warehouse_windows.iloc[i + 1]['window']), 'pickup_end'] = True


# Filter to show only start and end times
pickup_times = window_df[(window_df['pickup_start']) | (window_df['pickup_end'])]

# Display results
print(pickup_times[pickup_times['warehouse_id'] == 1615])


                 window  active  warehouse_id  pickup_start  pickup_end
18  2024-07-05 00:00:00    True          1615          True       False
278 2024-07-05 00:40:00   False          1615         False        True
343 2024-07-05 00:50:00    True          1615          True       False
408 2024-07-05 01:00:00   False          1615         False        True


2024-07-05 16:21:59,511| ERROR   | Socket exception: Operation timed out (60)


In [43]:
# Initialize lists to store results
results = []

# Iterate through each warehouse_id
for warehouse in window_df['warehouse_id'].unique():
    # Filter rows for the current warehouse_id and reset index for iteration
    warehouse_windows = window_df[window_df['warehouse_id'] == warehouse].reset_index(drop=True)
    i = 0
    while i < len(warehouse_windows):
        if warehouse_windows.iloc[i]['pickup_start']:
            # Find the corresponding pickup_end
            j = i + 1
            while j < len(warehouse_windows):
                if warehouse_windows.iloc[j]['pickup_end']:
                    # Append the result for this pair of pickup_start and pickup_end
                    results.append({
                        'warehouse_id': warehouse,
                        'pickup_start_time': warehouse_windows.iloc[i]['window'],
                        'pickup_end_time': warehouse_windows.iloc[j]['window']
                    })
                    i = j + 1  # Move to the next pickup_start
                    break
                j += 1
        else:
            i += 1

# Create DataFrame from results list
results_df = pd.DataFrame(results)

# Print the result
print(results_df)


     warehouse_id   pickup_start_time     pickup_end_time
0             329 2024-07-05 10:50:00 2024-07-05 11:20:00
1             370 2024-07-05 04:20:00 2024-07-05 04:30:00
2             370 2024-07-05 10:50:00 2024-07-05 11:30:00
3             370 2024-07-05 14:10:00 2024-07-05 14:20:00
4             370 2024-07-05 15:00:00 2024-07-05 15:10:00
..            ...                 ...                 ...
117          2122 2024-07-05 13:50:00 2024-07-05 15:00:00
118          2129 2024-07-05 11:50:00 2024-07-05 12:00:00
119          2129 2024-07-05 14:10:00 2024-07-05 14:20:00
120          2140 2024-07-05 11:10:00 2024-07-05 11:20:00
121          2147 2024-07-05 11:50:00 2024-07-05 12:00:00

[122 rows x 3 columns]
